# 1.0 Required Libraries

In [42]:
from app.configs.logging_file import logging
from urllib.parse import urlparse
from app.models.data_preprocessing import DataPreprocessing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os

In [2]:
path_dict = {
    "web_credibility_1000_url_ratings": os.path.join("app","datasets","webcredibility-1","webcredibility","web_credibility_1000_url_ratings.xls"),
    "web_credibility_expert_ratings_for_test_set": os.path.join("app","datasets","webcredibility-1","webcredibility","web_credibility_expert_ratings_for_test_set.xlsx"),
    
}
logging.info(f"__path_dict__ : {path_dict}")

2024-12-08 12:29:09 - 1426566399.py - Line: 6 - <module>() - INFO - __path_dict__ : {'web_credibility_1000_url_ratings': 'app/datasets/webcredibility-1/webcredibility/web_credibility_1000_url_ratings.xls', 'web_credibility_expert_ratings_for_test_set': 'app/datasets/webcredibility-1/webcredibility/web_credibility_expert_ratings_for_test_set.xlsx'}


In [3]:
if os.path.exists(path_dict['web_credibility_1000_url_ratings']):
    web_credibility_1000_url_ratings_df = pd.read_excel(path_dict['web_credibility_1000_url_ratings'])
else:
    logging.error(f"File not found at given path: {path_dict['web_credibility_1000_url_ratings']}")

if os.path.exists(path_dict['web_credibility_expert_ratings_for_test_set']):
    web_credibility_expert_ratings_for_test_set_df = pd.read_excel(path_dict['web_credibility_expert_ratings_for_test_set'])
else:
    logging.error(f"File not found at given path : {path_dict['web_credibility_expert_ratings_for_test_set']}")

In [4]:
logging.info(f"Shape of web_credibility_1000_url_ratings_df : {web_credibility_1000_url_ratings_df.shape}")
web_credibility_1000_url_ratings_df.head()

2024-12-08 12:29:10 - 3884070208.py - Line: 1 - <module>() - INFO - Shape of web_credibility_1000_url_ratings_df : (1000, 5)


,Topic,Query,Result Rank,URL,Likert Rating
0,Celebrities,adam lambert,1,http://en.wikipedia.org/wiki/Adam_Lambert,5
1,Celebrities,adam lambert,2,http://www.adamofficial.com/us/intro,4
2,Celebrities,adam lambert,3,http://www.adamofficial.com/us/home,4
3,Celebrities,adam lambert,4,http://www.thehollywoodgossip.com/2010/06/new-...,3
4,Celebrities,adam lambert,5,http://www.americanidol.com/contestants/season...,4


In [5]:
logging.info(f"shape of web_credibility_expert_ratings_for_test_set_df : {web_credibility_expert_ratings_for_test_set_df.shape}")
web_credibility_expert_ratings_for_test_set_df.head()

2024-12-08 12:29:11 - 1743633977.py - Line: 1 - <module>() - INFO - shape of web_credibility_expert_ratings_for_test_set_df : (40, 5)


,Topic,Query,URL,Rater ID,Likert Rating
0,Health,h1n1 vaccine side effects,http://canadiansforhealthfreedom.wordpress.com...,Medical Expert 1,1
1,Health,h1n1 vaccine side effects,http://canadiansforhealthfreedom.wordpress.com...,Medical Expert 2,1
2,Health,h1n1 vaccine side effects,http://health.blogs.foxnews.com/2009/10/26/tra...,Medical Expert 1,4
3,Health,h1n1 vaccine side effects,http://health.blogs.foxnews.com/2009/10/26/tra...,Medical Expert 2,2
4,Health,h1n1 vaccine side effects,http://revealedtruth.wordpress.com/2009/08/11/...,Medical Expert 1,1


In [6]:
def get_domain(url=None):
    if url:
        parsed_url = urlparse(url)  # Parse the URL
        return parsed_url.netloc  # Extract the domain (e.g., 'example.com')
    return None
def check_cached_pages_existance(domain=None):
    list_of_caches_files = os.listdir(os.path.join("app","datasets","webcredibility-1","webcredibility","cached_pages"))
    if domain in list_of_caches_files:
        return True
    else:
        return False
    
web_credibility_1000_url_ratings_df['domain'] = web_credibility_1000_url_ratings_df['URL'].apply(lambda url : get_domain(url = url))
web_credibility_1000_url_ratings_df['check_cached_pages_existance'] = web_credibility_1000_url_ratings_df['domain'].apply(lambda domain : 
                                                                                                                          check_cached_pages_existance(domain = domain))

In [7]:
web_credibility_1000_url_ratings_df['check_cached_pages_existance'].value_counts()

check_cached_pages_existance
True     950
False     50
Name: count, dtype: int64

## 2.0 **Content Features**
1. Word count 
2. Average sentence length
3. Keyword density
4. Number of outbound links
5. Presence of authoritative links (e.g., .gov, .edu)
6. Number of broken links
7. Sentiment analysis score
8. Frequency of grammatical errors



In [13]:
import os

def html_files_walk(path=None):
    html_files = []
    for dirpath, _, filenames in os.walk(path):
        for file in filenames:
            if file.endswith(".html"):  # Check if file has .html extension
                full_path = os.path.join(dirpath, file)
                html_files.append(full_path)
    return html_files

def get_html_files_path(dir_path=None):
    # Root path where HTML files are cached
    root_path = os.path.join("app", "datasets", "webcredibility-1", "webcredibility", "cached_pages")
    to_files_path = os.path.join(root_path,dir_path)
    
    if os.path.exists(to_files_path):
        list_of_path = html_files_walk(path=to_files_path)
    else:
        list_of_path = []

    return list_of_path
        
        
        
        


web_credibility_1000_url_ratings_df['html_walk_paths'] = web_credibility_1000_url_ratings_df['domain'].apply(
                                                                                lambda domain_name : get_html_files_path(dir_path = domain_name))

web_credibility_1000_url_ratings_df

,Topic,Query,Result Rank,URL,Likert Rating,domain,check_cached_pages_existance,html_walk_paths
0,Celebrities,adam lambert,1,http://en.wikipedia.org/wiki/Adam_Lambert,5,en.wikipedia.org,False,[]
1,Celebrities,adam lambert,2,http://www.adamofficial.com/us/intro,4,www.adamofficial.com,True,[app/datasets/webcredibility-1/webcredibility/...
2,Celebrities,adam lambert,3,http://www.adamofficial.com/us/home,4,www.adamofficial.com,True,[app/datasets/webcredibility-1/webcredibility/...
3,Celebrities,adam lambert,4,http://www.thehollywoodgossip.com/2010/06/new-...,3,www.thehollywoodgossip.com,True,[app/datasets/webcredibility-1/webcredibility/...
4,Celebrities,adam lambert,5,http://www.americanidol.com/contestants/season...,4,www.americanidol.com,True,[app/datasets/webcredibility-1/webcredibility/...
...,...,...,...,...,...,...,...,...
995,Politics,Tea Party,36,http://stlouisteaparty.com/,3,stlouisteaparty.com,True,[app/datasets/webcredibility-1/webcredibility/...
996,Politics,Tea Party,37,http://abcnews.go.com/Politics/tea-party-prote...,4,abcnews.go.com,True,[app/datasets/webcredibility-1/webcredibility/...
997,Politics,Tea Party,38,http://topics.politico.com/index.cfm/topic/Tea...,3,topics.politico.com,True,[app/datasets/webcredibility-1/webcredibility/...
998,Politics,Tea Party,39,http://www.nationwidechicagoteaparty.com/,3,www.nationwidechicagoteaparty.com,True,[app/datasets/webcredibility-1/webcredibility/...


In [18]:
web_credibility_1000_url_ratings_df['html_walk_paths'][4]

['app/datasets/webcredibility-1/webcredibility/cached_pages/www.americanidol.com/contestants/season_8/adam_lambert/index.html']

In [24]:
group_df

Topic                                                                 Celebrities
Query                                                                adam lambert
Result Rank                                                                     2
URL                                          http://www.adamofficial.com/us/intro
Likert Rating                                                                   4
domain                                                       www.adamofficial.com
check_cached_pages_existance                                                 True
html_walk_paths                 [app/datasets/webcredibility-1/webcredibility/...
Name: 1, dtype: object

In [44]:
def calculate_keyword_density(text, keyword='credibility'):
    words = text.lower().split()
    return words.count(keyword.lower()) / max(len(words), 1)

def extract_keyword_density_from_html(file_paths=None):
    
    try:
        
        features = {}
        keyword_density_counter =0
        for file_path in file_paths:
            
            # Read the HTML file
            with open(file_path, 'r', encoding='utf-8',errors='replace') as file:
                content = file.read()
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(content, 'lxml')
            
            keyword_density = calculate_keyword_density(soup.get_text())

            avg_sentence_length_counter = keyword_density_counter + keyword_density
        return avg_sentence_length_counter
    except Exception as e:
        logging.info(f"Error Found at extract_keyword_density_from_html : {file_paths}",exc_info=True)

In [45]:
web_credibility_1000_url_ratings_df['keyword_density'] = web_credibility_1000_url_ratings_df['html_walk_paths'].apply(lambda path : extract_keyword_density_from_html(file_paths = path))


/tmp/ipykernel_19915/584757829.py:13: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(content, 'lxml')


In [31]:
for index, group_df in web_credibility_1000_url_ratings_df.iterrows():
    print(f"Index: {index}")
    
    # Check the value of 'check_cached_pages_existance'
    if group_df['check_cached_pages_existance'] == True:
        print("Cached pages exist for this row.")
        # You can process the row further here if needed
        print(group_df['html_walk_paths'])
        html_walk_paths = group_df['html_walk_paths']

        extract_features_from_html(file_paths = html_walk_paths) 
        break


Index: 0
Index: 1
Cached pages exist for this row.
['app/datasets/webcredibility-1/webcredibility/cached_pages/www.adamofficial.com/us/home.html', 'app/datasets/webcredibility-1/webcredibility/cached_pages/www.adamofficial.com/us/intro.html']
word_count  : 574
word_count  : 159


In [27]:
html_walk_paths

['app/datasets/webcredibility-1/webcredibility/cached_pages/www.adamofficial.com/us/home.html',
 'app/datasets/webcredibility-1/webcredibility/cached_pages/www.adamofficial.com/us/intro.html']

# 3.0 **Design and Technical Features**
1. HTTPS usage (SSL certificate presence)
2. Number of ads or iframes
3. Mobile responsiveness
4. Flesch Reading Ease score
5. Page load time
6. Total page size



# 4.0 **Media Features**
1. Number of images
2. Number of videos
3. Presence of alt text for images




# 5. **Off-Page Features**
1. Domain age
2. Alexa rank or similar website ranking
3. Number of backlinks
4. Social media shares



# 6.0 **Aggregate Features**
1. Number of unique visitors
2. Average time spent on page
3. Bounce rate
4. Geographic diversity of visitors